In [1]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv("user_sessions_log.tsv", sep="\t")
dfs = df.shift(-1)

In [17]:
df

,user_id,session_id,timestamp,action,value
0,user_12150,1,1663707632,category,0.0
1,user_12150,1,1663707659,search,0.0
2,user_12150,1,1663707664,category,0.0
3,user_12150,1,1663707675,cart,44700.0
4,user_12150,1,1663707711,mainpage,0.0
...,...,...,...,...,...
5338912,user_12131,210,1697835575,search,0.0
5338913,user_12131,211,1697835580,search,0.0
5338914,user_10857,185,1697835596,mainpage,0.0
5338915,user_12131,211,1697835600,category,0.0


In [37]:
cnt = df.groupby(["user_id", "session_id", "timestamp"])["action"].apply(lambda x: int("mainpage" in set(x))).reset_index()

In [39]:
cnt.groupby(["user_id", "session_id"]).agg({"action": "sum"})

action
user_id    session_id        
user_10000 1                0
           2                0
           3                0
           4                1
           5                0
...                       ...
user_14999 195              2
           196              1
           197              0
           198              0
           199              1

[1004603 rows x 1 columns]

In [46]:
df[(df["user_id"] == "user_10184") & (df["session_id"] == 66)]

,user_id,session_id,timestamp,action,value
1923542,user_10184,66,1676061323,category,0.0
1923545,user_10184,66,1676061347,product,0.0
1923548,user_10184,66,1676061357,category,0.0
1923549,user_10184,66,1676061364,search,0.0
1923550,user_10184,66,1676061364,category,0.0
1923551,user_10184,66,1676061364,product,0.0


In [41]:
df.groupby(["user_id", "session_id", "timestamp"])["action"].apply("count").sort_values()

user_id     session_id  timestamp 
user_10000  1           1664060432    1
user_13327  93          1679844803    1
                        1679844802    1
                        1679844775    1
                        1679844753    1
                                     ..
user_10118  72          1676131141    3
user_12103  169         1689083576    3
user_14875  132         1684381005    3
user_10109  49          1671771923    3
user_10184  66          1676061364    3
Name: action, Length: 5302184, dtype: int64

In [3]:
def process(df: pd.DataFrame):
    cond = df.groupby(["user_id", "session_id"]).agg(
        timemin=("timestamp", "min"), timemax=("timestamp", "max")).reset_index()
    cond["dif"] = cond["timemax"] - cond["timemin"]
    cond2 = df.merge(cond, on=["user_id", "session_id"])
    cond2 = cond2[cond2["dif"] >= 1]
    tot = cond2.drop_duplicates(["user_id", "session_id"])
    mpage = cond2[(cond2["action"] == "mainpage") & (
        cond2["timestamp"] == cond2["timemin"])].drop_duplicates(["user_id", "session_id"])
    return len(mpage) / len(tot) * 100


In [62]:
cond = df.groupby(["user_id", "session_id"]).agg(timemin=("timestamp", "min"), timemax=("timestamp", "max")).reset_index()
cond["dif"] = cond["timemax"] - cond["timemin"]
cond2 = df.merge(cond, on=["user_id", "session_id"])
cond2 = cond2[cond2["dif"] >= 1]
tot = cond2.drop_duplicates(["user_id", "session_id"])
mpage = cond2[(cond2["action"] == "mainpage") & (cond2["timestamp"] == cond2["timemin"])].drop_duplicates(["user_id", "session_id"])

In [64]:
len(mpage) / len(tot) * 100

21.387161883434498

In [60]:
len(mpage) / len(tot) * 100

49.7311273498733

In [56]:
df.drop_duplicates(["user_id", "session_id"])

,user_id,session_id,timestamp,action,value
0,user_12150,1,1663707632,category,0.0
6,user_14808,1,1663707789,search,0.0
8,user_10444,1,1663707800,search,0.0
16,user_10444,2,1663707834,search,0.0
18,user_10472,1,1663707843,search,0.0
...,...,...,...,...,...
5338900,user_10939,220,1697835479,category,0.0
5338904,user_12131,210,1697835521,search,0.0
5338906,user_13624,203,1697835535,mainpage,0.0
5338908,user_10857,185,1697835544,search,0.0


In [44]:
pd.merge(df, cond, on=["user_id", "session_id"])

,user_id,session_id,timestamp,action,value,timemin,timemax,dif
0,user_12150,1,1663707632,category,0.0,1663707632,1663707734,102
1,user_12150,1,1663707659,search,0.0,1663707632,1663707734,102
2,user_12150,1,1663707664,category,0.0,1663707632,1663707734,102
3,user_12150,1,1663707675,cart,44700.0,1663707632,1663707734,102
4,user_12150,1,1663707711,mainpage,0.0,1663707632,1663707734,102
...,...,...,...,...,...,...,...,...
5338912,user_10857,185,1697835563,product,0.0,1697835544,1697835596,52
5338913,user_10857,185,1697835596,mainpage,0.0,1697835544,1697835596,52
5338914,user_12131,211,1697835580,search,0.0,1697835580,1697835611,31
5338915,user_12131,211,1697835600,category,0.0,1697835580,1697835611,31


In [53]:
from datetime import datetime

In [54]:
datetime.fromtimestamp(1663707843)

datetime.datetime(2022, 9, 21, 0, 4, 3)

In [55]:
datetime.fromtimestamp(1663707844)

datetime.datetime(2022, 9, 21, 0, 4, 4)